In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

# Usable Code

## Scratch Work

In [2]:
!pwd

/Users/jseemayer/Documents/Me/Metis/NBA-Over-Under-Predictor


In [3]:
cd csv

/Users/jseemayer/Documents/Me/Metis/NBA-Over-Under-Predictor/csv


In [4]:
df = pd.read_csv('nba odds 2017-18.csv')

In [5]:
df.head()

,Date,Rot,VH,Team,1st,2nd,3rd,4th,Final,Open,...,2H,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21
0,1017,501,V,Boston,19,19,33,28,99,214.5,...,1.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1017,502,H,Cleveland,29,25,18,30,102,4,...,110,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1017,503,V,Houston,34,28,26,34,122,232,...,115.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1017,504,H,GoldenState,35,36,30,20,121,9,...,2.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1018,701,V,Brooklyn,30,33,35,33,131,212,...,112.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2624 entries, 0 to 2623
Data columns (total 22 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Date         2624 non-null   int64  
 1   Rot          2624 non-null   int64  
 2   VH           2624 non-null   object 
 3   Team         2624 non-null   object 
 4   1st          2624 non-null   int64  
 5   2nd          2624 non-null   int64  
 6   3rd          2624 non-null   int64  
 7   4th          2624 non-null   int64  
 8   Final        2624 non-null   int64  
 9   Open         2624 non-null   object 
 10  Close        2624 non-null   object 
 11  ML           2624 non-null   int64  
 12  2H           2624 non-null   object 
 13  Unnamed: 13  0 non-null      float64
 14  Unnamed: 14  0 non-null      float64
 15  Unnamed: 15  0 non-null      float64
 16  Unnamed: 16  0 non-null      float64
 17  Unnamed: 17  0 non-null      float64
 18  Unnamed: 18  0 non-null      float64
 19  Unname

In [7]:
df = df.iloc[:,:13]

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2624 entries, 0 to 2623
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Date    2624 non-null   int64 
 1   Rot     2624 non-null   int64 
 2   VH      2624 non-null   object
 3   Team    2624 non-null   object
 4   1st     2624 non-null   int64 
 5   2nd     2624 non-null   int64 
 6   3rd     2624 non-null   int64 
 7   4th     2624 non-null   int64 
 8   Final   2624 non-null   int64 
 9   Open    2624 non-null   object
 10  Close   2624 non-null   object
 11  ML      2624 non-null   int64 
 12  2H      2624 non-null   object
dtypes: int64(8), object(5)
memory usage: 266.6+ KB


Just trying on 2 games

In [9]:
df1 = df.iloc[:4,:]

In [10]:
df1

,Date,Rot,VH,Team,1st,2nd,3rd,4th,Final,Open,Close,ML,2H
0,1017,501,V,Boston,19,19,33,28,99,214.5,216,166,1.5
1,1017,502,H,Cleveland,29,25,18,30,102,4,4.5,-195,110
2,1017,503,V,Houston,34,28,26,34,122,232,231.5,364,115.5
3,1017,504,H,GoldenState,35,36,30,20,121,9,9,-470,2.5


In [11]:
pd.set_option("display.max_columns", 101)
pd.set_option("display.max_rows", 300)

In [12]:
df.shape[0]

2624

In [13]:
mylist = []
for i in range(1, int(df.shape[0]/2 + 1)):
    mylist.append(i)
    mylist.append(i)
print(mylist[0],mylist[-1])

1 1312


In [14]:
len(range(1,int(df.shape[0]/2 + 1)))

1312

In [15]:
df['game_id'] = mylist

In [16]:
df

,Date,Rot,VH,Team,1st,2nd,3rd,4th,Final,Open,Close,ML,2H,game_id
0,1017,501,V,Boston,19,19,33,28,99,214.5,216,166,1.5,1
1,1017,502,H,Cleveland,29,25,18,30,102,4,4.5,-195,110,1
2,1017,503,V,Houston,34,28,26,34,122,232,231.5,364,115.5,2
3,1017,504,H,GoldenState,35,36,30,20,121,9,9,-470,2.5,2
4,1018,701,V,Brooklyn,30,33,35,33,131,212,216.5,135,112.5,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2619,603,504,H,GoldenState,32,27,31,32,122,12.5,11.5,-600,3.5,1310
2620,606,505,V,GoldenState,28,24,31,27,110,5,3,-150,5,1311
2621,606,506,H,Cleveland,29,29,23,21,102,217.5,216,130,107,1311
2622,608,507,V,GoldenState,34,27,25,22,108,5.5,4.5,-180,1,1312


In [17]:
df = df.merge(df, on='game_id', suffixes=(None,'second_row'))

In [18]:
df

,Date,Rot,VH,Team,1st,2nd,3rd,4th,Final,Open,Close,ML,2H,game_id,Datesecond_row,Rotsecond_row,VHsecond_row,Teamsecond_row,1stsecond_row,2ndsecond_row,3rdsecond_row,4thsecond_row,Finalsecond_row,Opensecond_row,Closesecond_row,MLsecond_row,2Hsecond_row
0,1017,501,V,Boston,19,19,33,28,99,214.5,216,166,1.5,1,1017,501,V,Boston,19,19,33,28,99,214.5,216,166,1.5
1,1017,501,V,Boston,19,19,33,28,99,214.5,216,166,1.5,1,1017,502,H,Cleveland,29,25,18,30,102,4,4.5,-195,110
2,1017,502,H,Cleveland,29,25,18,30,102,4,4.5,-195,110,1,1017,501,V,Boston,19,19,33,28,99,214.5,216,166,1.5
3,1017,502,H,Cleveland,29,25,18,30,102,4,4.5,-195,110,1,1017,502,H,Cleveland,29,25,18,30,102,4,4.5,-195,110
4,1017,503,V,Houston,34,28,26,34,122,232,231.5,364,115.5,2,1017,503,V,Houston,34,28,26,34,122,232,231.5,364,115.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5243,606,506,H,Cleveland,29,29,23,21,102,217.5,216,130,107,1311,606,506,H,Cleveland,29,29,23,21,102,217.5,216,130,107
5244,608,507,V,GoldenState,34,27,25,22,108,5.5,4.5,-180,1,1312,608,507,V,GoldenState,34,27,25,22,108,5.5,4.5,-180,1
5245,608,507,V,GoldenState,34,27,25,22,108,5.5,4.5,-180,1,1312,608,508,H,Cleveland,25,27,13,20,85,216,216.5,155,107
5246,608,508,H,Cleveland,25,27,13,20,85,216,216.5,155,107,1312,608,507,V,GoldenState,34,27,25,22,108,5.5,4.5,-180,1


In [24]:
df = df[1::4]

In [25]:
df

,Date,Rot,VH,Team,1st,2nd,3rd,4th,Final,Open,Close,ML,2H,game_id,Datesecond_row,Rotsecond_row,VHsecond_row,Teamsecond_row,1stsecond_row,2ndsecond_row,3rdsecond_row,4thsecond_row,Finalsecond_row,Opensecond_row,Closesecond_row,MLsecond_row,2Hsecond_row
1,1017,501,V,Boston,19,19,33,28,99,214.5,216,166,1.5,1,1017,502,H,Cleveland,29,25,18,30,102,4,4.5,-195,110
5,1017,503,V,Houston,34,28,26,34,122,232,231.5,364,115.5,2,1017,504,H,GoldenState,35,36,30,20,121,9,9,-470,2.5
9,1018,701,V,Brooklyn,30,33,35,33,131,212,216.5,135,112.5,3,1018,702,H,Indiana,29,36,41,34,140,3.5,3,-155,1.5
13,1018,703,V,Philadelphia,23,36,23,33,115,215.5,217.5,240,108.5,4,1018,704,H,Washington,32,24,34,30,120,6.5,6.5,-300,6
17,1018,705,V,Miami,37,18,23,31,109,3,3.5,-170,4,5,1018,706,H,Orlando,32,26,30,28,116,207,205.5,145,100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5229,528,511,V,GoldenState,19,24,33,25,101,4,6,-265,9,1308,528,512,H,Houston,24,30,15,23,92,210,208,218,103.5
5233,531,501,V,Cleveland,30,26,22,29,114,215.5,218,664,107.5,1309,531,502,H,GoldenState,29,27,28,23,124,11.5,13,-1000,9.5
5237,603,503,V,Cleveland,28,18,34,23,103,216,216,447,106,1310,603,504,H,GoldenState,32,27,31,32,122,12.5,11.5,-600,3.5
5241,606,505,V,GoldenState,28,24,31,27,110,5,3,-150,5,1311,606,506,H,Cleveland,29,29,23,21,102,217.5,216,130,107


## Scratch Work Graveyard

In [ ]:
#tried self-merging on game_id, but this is only applicable if in different columns
#df1 = df1.merge(
#            right=df1[opp_pull_cols],
#            left_on=["game_id", "team"],
#            right_on=["game_id", "opp"],
#            suffixes=[None, "_opp"],
#        )

In [22]:
#tried to use drop_duplicates method but can only keep first or last
#df.drop_duplicates(subset=['game_id'], keep='second')

ValueError: keep must be either "first", "last" or False